In [33]:
import os
import hw
import numpy as np
import env_ingredient_add as env
from hw import PoseTrajectorySource

def get_scene():
    if os.getcwd() == "/datasets/_deepnote_work/manipulation/project": 
        #scene = open("/work/manipulation/project/objects/environment_setup.yaml")
        scene = env.get_environment_set_up(no_scene=True,include_driver=True)
        xmls = [os.getcwd() + "/package.xml", "/work/manipulation/project/package.xml"]
    else:
        #scene = open("objects/environment_setup.yaml") # local setup
        scene = env.get_environment_set_up(no_scene=False,include_driver=True)
        xmls = [os.getcwd() + "/package.xml"]

    return scene

In [7]:
scenario = hw.LoadScenario(data=get_scene())
meshcat = hw.StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [8]:
# at present, trajectory blank doesn't work
diagram,sim = hw.init_diagram(meshcat,scenario)
# hw.print_diagram(diagram)

context = diagram.CreateDefaultContext()
traj = hw.create_small_trajectory(diagram,meshcat,context)

meshcat.Delete()
diagram,sim = hw.init_diagram(meshcat,scenario,traj)
controller = diagram.GetSubsystemByName("diff_ik_integrator")


In [13]:
hw.fix_base_pos(controller.get_parameters(), 3*[True])
hw.create_small_trajectory(diagram,meshcat)

hw.run_simulation(meshcat,sim,20)

In [ ]:
def traj_move_to_bowl_0(
        diagram,
        context=None,
        move_time=10 # seconds
    ) -> PoseTrajectorySource:
    if context is None:
        context = diagram.CreateDefaultContext()

    start_time = context.get_time()

    # get a starting point
    X_WGinit = hw.get_X_WG(diagram,context)

    # define a goal pose
    goal_pose = hw.RigidTransform(
        R=hw.RotationMatrix(),
        p=np.array(env.bowl_0) + np.array([0.25, 0, 0.25])
    )

    poses = hw.PiecewisePose.MakeLinear([start_time, start_time + move_time], [X_WGinit, goal_pose])

    return poses


def plot_traj(meshcat,traj):

    # reconstruct the full trajectory
    poses = np.array(traj.get_times())
    print(poses)

    # for i, frame in enumerate(poses):
    #     hw.AddMeshcatTriad(meshcat, X_PT=frame, path="frame"+str(i))

my_context = sim.get_context()

new_traj = traj_move_to_bowl_0(diagram,my_context)

# problem: we're missing a way to actually pass new trajectories into the robot.
pose_source = diagram.GetSubsystemByName("hw.PoseTrajectorySource@00007fc3283167c0")

## scratch code

### gets the name of the robot's joints

In [ ]:

# plant = diagram.GetSubsystemByName("station").GetSubsystemByName("plant")

# for index in plant.GetJointIndices():
#     print(plant.get_joint(index).name())

iiwa_base_x
iiwa_base_y
iiwa_base_z
iiwa_joint_1
iiwa_joint_2
iiwa_joint_3
iiwa_joint_4
iiwa_joint_5
iiwa_joint_6
iiwa_joint_7
iiwa_joint_ee
tool0_joint
left_finger_sliding_joint
right_finger_sliding_joint
iiwa_link_7_welds_to_body


In [ ]:


print(controller.get_parameters().get_joint_position_limits())

print(f"after running: {controller.get_parameters().get_joint_position_limits()}")

hw.run_simulation(meshcat,sim,10)